In [ ]:
# Importing modules
import pandas as pd
import os


# Read data into papers
papers = pd.read_csv('MoviesPlot.csv')

# Print head
papers.head()

,Year,Plot,imdbRating,imdbID
0,2001.0,An English Duke from 1876 is inadvertedly drag...,6.4,tt0035423
1,2020.0,The story of a man whose wife has committed su...,6.3,tt0062336
2,2018.0,A Hollywood director emerges from semi-exile w...,6.7,tt0069049
3,2018.0,"Village born Lalloo re-locates to Bombay, and ...",6.1,tt0069204
4,2001.0,A group of Javanese villagers are driven to re...,7.3,tt0079644


** **
#### Step 2: Data Cleaning
** **

Since the goal of this analysis is to perform topic modeling, we will solely focus on the text data from each paper, and drop other metadata columns

In [ ]:
# Remove the columns
papers = papers.drop(columns=['imdbRating','imdbID', 'Year'], axis=1)

# sample only 100 papers
papers = papers.sample(100)

# Print out the first rows of papers
papers.head()

,Plot
80547,"A tough, cute woman and a cyborg arrive at a d..."
87543,Newlywed Kath takes off on a spur-of-the-momen...
29568,The story revolves around the clash between Ay...
28595,"Ernesto, a seventy-eight-year-old Uruguayan ph..."
85032,Families and healthcare workers find themselve...


##### Remove punctuation/lower casing

Next, let’s perform a simple preprocessing on the content of paper_text column to make them more amenable for analysis, and reliable results. To do that, we’ll use a regular expression to remove any punctuation, and then lowercase the text

In [ ]:
# Load the regular expression library
import re

# Remove punctuation
papers['paper_text_processed'] = papers['Plot'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
papers['paper_text_processed'] = papers['paper_text_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
papers['paper_text_processed'].head()

80547    a tough cute woman and a cyborg arrive at a de...
87543    newlywed kath takes off on a spur-of-the-momen...
29568    the story revolves around the clash between ay...
28595    ernesto a seventy-eight-year-old uruguayan pho...
85032    families and healthcare workers find themselve...
Name: paper_text_processed, dtype: object

##### Tokenize words and further clean-up text

Let’s tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.

In [ ]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['tough', 'cute', 'woman', 'and', 'cyborg', 'arrive', 'at', 'depopulated', 'post', 'ww', 'times', 'sq', 'they', 'leave', 'the', 'ruins', 'of', 'nyc', 'with', 'on', 'secret', 'mission']


** **
#### Step 3: Phrase Modeling: Bigram and Trigram Models
** **

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring. Some examples in our example are: 'back_bumper', 'oil_leakage', 'maryland_college_park' etc.

Gensim's Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold.

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


#### Remove Stopwords, Make Bigrams and Lemmatize

The phrase models are ready. Let’s define the functions to remove the stopwords, make trigrams and lemmatization and call them sequentially.

In [ ]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

Let's call the functions in order.

In [ ]:
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 4.9 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
#!pip install spacy
import spacy


# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['tough', 'cute', 'woman', 'cyborg', 'depopulate', 'leave', 'ruin', 'secret', 'mission']


** **
#### Step 4: Data transformation: Corpus and Dictionary
** **

The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.

In [ ]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]


** **
#### Step 5: Base Model 
** **

We have everything required to train the base LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well. Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior (we'll use default for the base model).

chunksize controls how many documents are processed at a time in the training algorithm. Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory.

passes controls how often we train the model on the entire corpus (set to 10). Another word for passes might be "epochs". iterations is somewhat technical, but essentially it controls how often we repeat a particular loop over each document. It is important to set the number of "passes" and "iterations" high enough.

In [ ]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

** **
The above LDA model is built with 10 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

You can see the keywords for each topic and the weightage(importance) of each keyword using `lda_model.print_topics()`

In [ ]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.021*"life" + 0.011*"relationship" + 0.011*"people" + 0.011*"turn" + '
  '0.011*"friend" + 0.011*"work" + 0.011*"city" + 0.011*"help" + 0.011*"son" + '
  '0.011*"haruhiko"'),
 (1,
  '0.016*"life" + 0.016*"young" + 0.011*"family" + 0.011*"man" + 0.011*"win" + '
  '0.011*"change" + 0.011*"grandfather" + 0.011*"girl" + 0.011*"mother" + '
  '0.011*"mystery"'),
 (2,
  '0.015*"dream" + 0.012*"year" + 0.012*"home" + 0.012*"love" + 0.012*"life" + '
  '0.008*"man" + 0.008*"young" + 0.008*"past" + 0.008*"make" + 0.008*"back"'),
 (3,
  '0.016*"brother" + 0.016*"go" + 0.012*"mother" + 0.012*"get" + 0.008*"try" + '
  '0.008*"horror" + 0.008*"end" + 0.008*"movie" + 0.008*"protest" + '
  '0.008*"teen"'),
 (4,
  '0.011*"love" + 0.011*"role" + 0.011*"story" + 0.011*"come" + 0.011*"girl" + '
  '0.011*"long" + 0.011*"captive" + 0.006*"comedy" + 0.006*"town" + '
  '0.006*"resort"'),
 (5,
  '0.015*"bad" + 0.010*"world" + 0.010*"take" + 0.010*"thing" + 0.010*"side" + '
  '0.010*"lead" + 0.010*"city

#### Compute Model Perplexity and Coherence Score

Let's calculate the baseline coherence score

In [ ]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.4952439328994451


** **
#### Step 6: Hyperparameter tuning
** **
First, let's differentiate between model hyperparameters and model parameters :

- `Model hyperparameters` can be thought of as settings for a machine learning algorithm that are tuned by the data scientist before training. Examples would be the number of trees in the random forest, or in our case, number of topics K

- `Model parameters` can be thought of as what the model learns during training, such as the weights for each word in a given topic.

Now that we have the baseline coherence score for the default LDA model, let's perform a series of sensitivity tests to help determine the following model hyperparameters: 
- Number of Topics (K)
- Dirichlet hyperparameter alpha: Document-Topic Density
- Dirichlet hyperparameter beta: Word-Topic Density

We'll perform these tests in sequence, one parameter at a time by keeping others constant and run them over the two difference validation corpus sets. We'll use `C_v` as our choice of metric for performance comparison 

In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

Let's call the function, and iterate it over the range of topics, alpha, and beta parameter values

In [ ]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()


100%|██████████| 540/540 [09:57<00:00,  1.11s/it]
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)

100%|██████████| 540/540 [09:39<00:00,  1.07s/it]


** **
#### Step 7: Final Model
** **

Based on external evaluation (Code to be added from Excel based analysis), let's train the final model with parameters yielding highest coherence score

In [ ]:
num_topics = 7

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.31)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [ ]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.015*"life" + 0.007*"meet" + 0.007*"former" + 0.007*"man" + 0.007*"take" + '
  '0.005*"turn" + 0.005*"run" + 0.005*"son" + 0.005*"capture" + '
  '0.005*"friend"'),
 (1,
  '0.010*"man" + 0.010*"life" + 0.008*"family" + 0.005*"love" + 0.005*"woman" '
  '+ 0.005*"back" + 0.005*"young" + 0.005*"change" + 0.005*"true" + '
  '0.005*"win"'),
 (2,
  '0.009*"love" + 0.009*"dream" + 0.007*"life" + 0.007*"young" + 0.005*"wave" '
  '+ 0.005*"mystery" + 0.005*"see" + 0.005*"mother" + 0.005*"girl" + '
  '0.005*"husband"'),
 (3,
  '0.008*"brother" + 0.008*"story" + 0.006*"get" + 0.006*"steal" + 0.006*"go" '
  '+ 0.006*"mother" + 0.006*"try" + 0.004*"police" + 0.004*"clash" + '
  '0.004*"protest"'),
 (4,
  '0.010*"time" + 0.009*"year" + 0.009*"story" + 0.009*"day" + 0.007*"girl" + '
  '0.005*"child" + 0.005*"comedy" + 0.005*"captive" + 0.005*"long" + '
  '0.005*"woman"'),
 (5,
  '0.012*"old" + 0.010*"year" + 0.008*"life" + 0.008*"bad" + 0.008*"lead" + '
  '0.008*"thing" + 0.005*"live" + 0.005

** **
#### Step 8: Visualize Results
** **

In [ ]:
#!pip install pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join('ldavis_tuned_'+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, 'ldavis_tuned_'+ str(num_topics) +'.html')

LDAvis_prepared

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.078822  0.011780       1        1  19.688343
2      0.045956  0.068745       2        1  15.550110
0      0.035183 -0.031149       3        1  15.194956
4      0.000793 -0.032033       4        1  13.305958
5      0.016336 -0.043411       5        1  12.210289
6     -0.022728  0.018677       6        1  12.069262
1      0.003281  0.007392       7        1  11.981081, topic_info=       Term      Freq     Total Category  logprob  loglift
43      old  3.000000  3.000000  Default  30.0000  30.0000
30    story  6.000000  6.000000  Default  29.0000  29.0000
172    life  9.000000  9.000000  Default  28.0000  28.0000
61     time  2.000000  2.000000  Default  27.0000  27.0000
383    film  3.000000  3.000000  Default  26.0000  26.0000
..      ...       ...       ...      ...      ...      ...
22     come  0.492954  2.673409   Topic7  -5.8073   0.4311
27   senior  0.492954  1.774797   Topic7  -5.8073   0.8408
30    story  0.492954  6.818549   Topic7  -5.8073  -0.5051
33   become  0.492954  3.845695   Topic7  -5.8073   0.0675
37     face  0.492954  2.589141   Topic7  -5.8073   0.4632

[369 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
133       4  0.767891      acapulco
9         3  0.749343  accidentally
64        3  0.576138     accompany
64        4  0.576138     accompany
354       1  0.562919       account
...     ...       ...           ...
82        2  0.199772         young
82        4  0.199772         young
82        6  0.199772         young
82        7  0.199772         young
676       3  0.565634        zombie

[318 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 5, 6, 7, 2])

** **
#### Closing Notes

We started with understanding why evaluating the topic model is essential. Next, we reviewed existing methods and scratched the surface of topic coherence, along with the available coherence measures. Then we built a default LDA model using Gensim implementation to establish the baseline coherence score and reviewed practical ways to optimize the LDA hyperparameters.

Hopefully, this article has managed to shed light on the underlying topic evaluation strategies, and intuitions behind it.

** **
#### References:
1. http://qpleple.com/perplexity-to-evaluate-topic-models/
2. https://www.amazon.com/Machine-Learning-Probabilistic-Perspective-Computation/dp/0262018020
3. https://papers.nips.cc/paper/3700-reading-tea-leaves-how-humans-interpret-topic-models.pdf
4. https://github.com/mattilyra/pydataberlin-2017/blob/master/notebook/EvaluatingUnsupervisedModels.ipynb
5. https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
6. http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf
7. http://palmetto.aksw.org/palmetto-webapp/